#### Solutions to the problems of Chapter 9
### Priority Queues and Heaps

### helper classes 

In [1]:
class Empty(Exception):
    pass 
class _DoublyLinkedBase:
    class _Node:
        __slots__='_element', '_prev', '_next'
        
        def __init__(self, element, prev, nxt):
            self._element = element 
            self._prev = prev
            self._next = nxt 
            
    def __init__(self):
        self._header = self._Node(None, None, None)
        self._trailer = self._Node(None, None, None)
        self._header._next = self._trailer 
        self._trailer._prev = self._header
        self._size = 0 
        
    def __len__(self): 
        return self._size 
    
    def is_empty(self):
        return self._size == 0 
    
    def _insert_between(self, e, predecessor, successor):
        newest = self._Node(e, predecessor, successor)
        predecessor._next = newest
        successor._prev = newest 
        self._size += 1
        return newest 
    
    def _delete_node(self, node):
        predecessor = node._prev 
        successor = node._next 
        predecessor._next = successor 
        successor._prev = predecessor 
        self._size -= 1
        element = node._element 
        node._prev = node._next = node._element = None
        return element 

class PositionalList(_DoublyLinkedBase):
    def __init__(self):
        super().__init__() 
        
    class Position: 
        def __init__(self, container, node):
            self._container = container 
            self._node = node 
            
        def element(self):
            return self._node._element 
        def __eq__(self, other): 
            return type(other) is type(self) and other._node is self._node
        def __ne__(self, other):
            return not(self==other)
    
    def _validate(self, p):
        if not isinstance(p, self.Position):
            raise TypeError('p must be proper position type')
        if p._container is not self:
            raise ValueError('p does not belong to this type')
        if p._node._next is None:
            raise ValueError('p is no longer valid')
        return p._node 
    
    def _make_position(self, node):
        if (node is self._header) or (node is self._trailer):
            return None
        else:
            return self.Position(self, node)
        
    def first(self):
        return self._make_position(self._header._next)
    
    def last(self):
        return self._make_position(self._trailer._prev)
    
    def before(self, p):
        node = self._validate(p)
        return self._make_position(node._prev)
    
    def after(self, p):
        node = self._validate(p)
        return self._make_position(node._next)
    
    def __iter__(self):
        cursor = self.first() 
        while cursor is not None:
            yield cursor.element()
            cursor = self.after(cursor)
            
    def _insert_between(self, e, predecessor, successor):
        node = super()._insert_between(e, predecessor, successor)
        return self._make_position(node)
    
    def add_first(self, e):
        return self._insert_between(e, self._header, self._header._next)
    
    def add_last(self, e):
        return self._insert_between(e, self._trailer._prev, self._trailer)
    
    def add_before(self, p, e):
        original = self._validate(p)
        return self._insert_between(e, original._prev, original)
    
    def add_after(self, p, e):
        original = self._validate(p)
        return self._insert_between(e, original, original._next)
    
    def delete(self, p):
        original = self._validate(p)
        return self._delete_node(original)
    
    def replace(self, p, e):
        original = self._validate(p)
        old_value = original._element 
        original._element = e
        return old_value

#### Implementation to the Priority Queue ADT,  UnsortedPriorityQueue   

In [2]:
class PriorityQueueBase:
    class _Item:
        __slots__= '_key', '_value'
        def __init__(self, key, value):
            self._key = key
            self._value = value 
        def __lt__(self, other):
            return self._key < other._key 
    def is_empty(self):
        return len(self) ==0 

In [3]:
class UnsortedPriorityQueue(PriorityQueueBase): 
    def __init__(self):
        self._data = PositionalList() 
    
    def __len__(self):
        return len(self._data)
    
    def add(self, key, value):
        self._data.add_last(self._Item(key, value))
    
    def _find_min(self):
        if self.is_empty():
            raise Empty("Empty Prioirty Queue")
        minimum = self._data.first() 
        current = self._data.after(minimum)
        while current is not None:
            if current.element()< minimum.element():
                minimum = current
            current = self._data.after(current)
        return minimum 
    
    def min(self):
        item = self._find_min().element() 
        return (item._key, item._value) 
    
    def remove_min(self):
        minimum = self._find_min()
        item = self._data.delete(minimum) 
        return (item._key, item._value) 

In [4]:
class SortedPriorityQueue(PriorityQueueBase):
    def __init__(self):
        self._data = PositionalList() 
    def __len__(self):
        return len(self._data)
    def add(self, key, value):
        new = self._Item(key, value)
        current = self._data.last() 
        while current is not None and new<current.element(): 
            current = self._data.before(current)
        if current is None: 
            self._data.add_first(new)
        else:
            self._data.add_after(current, new)
            
    def min(self):
        if self.is_empty():
            raise Empty("Empty Priority Queue")
        min_item = self._data.first().element()
        return (min_item._key, min_item._value)
    
    def remove_min(self):
        if self.is_empty():
            raise Empty("Empty Priority Queue")
        min_item = self._data.delete(self._data.first())
        return (min_item._key, min_item._value)

#### Heap Data structure 

In [39]:
class HeapPriorityQueue(PriorityQueueBase):
    def __init__(self, contents = ()): 
        self._data = [self._Item(k, v) for k, v in contents]
        if len(self._data)>1:
            self._heapify() 
            
    def __len__(self):
        return len(self._data) 
    
    def _heapify(self): 
        start = self.parent(len(self._data)-1)  
        for j in range(start, -1, -1):
            self._downheap(j)
    
    def add(self, key, value):
        self._data.append(self._Item(key, value))
        self._upheap(len(self._data)-1)
    
    def min(self):
        if self.is_empty():
            raise Empty("Empty Priority Queue!")
        mini = self._data[0]
        return (mini._key, mini._value)
    
    def remove_min(self):
        if self.is_empty():
            raise Empty("Empty Priority Queue!")
        self._swap(0, len(self._data)-1)
        mini = self._data.pop()
        self._downheap(0)
        return (mini._key, mini._value) 
    
    def _parent(self, j):
        return (j-1)//2 
    
    def _left(self, j):
        return (2*j) + 1 
    
    def _right(self, j):
        return (2*j) + 2 
    
    def _has_left(self, j):
        return self._left(j) < len(self._data)
    
    def _has_right(self, j):
        return self._right(j) < len(self._data)
    
    def _swap(self, i, j):
        self._data[i], self._data[j] = self._data[j], self._data[i]
    
    def _upheap(self, j):
        parent = self._parent(j)
        if j>0 and (self._data[j]<self._data[parent]): 
            self._swap(j, parent)
            self._upheap(parent)
    
    def _downheap(self, j):
        if self._has_left(j):
            small_child = self._left(j)
            if self._has_right(j):
                right_child = self._right(j)
                if self._data[right_child]<self._data[small_child]:
                    small_child = right_child
            
            if self._data[small_child]<self._data[j]: 
                self._swap(j, small_child)
                self._downheap(small_child)
                
    def __repr__(self): 
        return ''.join("("+str(item._key)+","+str(item._value)+")" for item in self._data)  

In [40]:
class AdaptableHeapPriorityQueue(HeapPriorityQueue):
    class Locator(HeapPriorityQueue._Item):
        __slot__ = '_index'
        def __init__(self, k, v, j):
            super().__init__(k, v)
            self._index = j
    
    def _swap(self, i, j):
        super()._swap(i, j)
        self._data[i]._index = i
        self._data[j]._index = j
        
    def _bubble(self, j):
        if j>0 and self._data[j] < self._data[self._parent(j)]:
            self._upheap(j)
        else:
            self._downheap(j)
            
    def add(self, key, value):
        token = self.Locator(key, value, len(self._data))
        self._data.append(token)
        self._upheap(len(self._data)-1)
        return token
    
    def update(self, loc, newkey, newval):
        j = loc._index 
        if not (0 <= j <len(self) and self._data[j] is loc):
            raise ValueError("InValid locator")
        loc._key = newkey
        loc._value = newval 
        self._bubble(j)
        
    def remove(self, loc):
        j = loc._index 
        if not(0<=j<len(self) and self._data[j]is loc):
            raise ValueError("Invalid locator")
        if j==len(self)-1:
            self._pop()
        else:
            self._swap(j, len(self)-1)
            self._data.pop()
            self._bubble(j)
        return(loc._key, loc._value)
    

***Reinforcement Problems***

R-9.5 The min method for the UnsortedPriorityQueue class executes in O(n)time, as analyzed in Table 9.2. Give a simple modification to the class so that min runs in O(1) time. Explain any necessary modifications to other methods of the class.

this can be done if we have a variable that keeps track of the min value with each addition. It checks if the newly added value is less than or equal to the current min. if less then it stores it. This variable can be accessed in O(1) time. 

R-9.6 Can you adapt your solution to the previous problem to make remove_min run in O(1) time for the UnsortedPriorityQueue class? Explain your answer.

I don't think so because we need to find the min each time we do a removal. 

R-9.12 Consider a situation in which a user has numeric keys and wishes to have a priority queue that is maximum-oriented. How could a standard (minoriented) priority queue be used for such a purpose?

replace each key with the negative value of that key. 

***Creativity Problems*** 

C-9.26 Show how to implement the stack ADT using only a priority queue and one additional integer instance variable.

In [41]:
class PQStack:
    counter = 0 
    def __init__(self):
        self._pq = HeapPriorityQueue()
        self._count = 0 
        
    def push(self, e): 
        self._count -= 1 
        self._pq.add(self._count, e)
        
    def pop(self):
        return self._pq.remove_min()[1]
    
    def top(self):
        return self._pq.min()[1]

In [42]:
stack = PQStack() 
stack.push("a")
stack.push("b")
stack.push("c")
stack.push("d")

print(stack.top())
print(stack.pop())
print(stack.pop())
print(stack.pop())
print(stack.pop())

d
d
c
b
a


C-9.27 Show how to implement the FIFO queue ADT using only a priority queue and one additional integer instance variable.

In [43]:
class PQFIFO:
    counter = 0 
    def __init__(self):
        self._pq = HeapPriorityQueue()
        self._count = 0 
        
    def enqueue(self, e): 
        self._count += 1 
        self._pq.add(self._count, e)
        
    def dequeue(self):
        return self._pq.remove_min()[1]
    
    def top(self):
        return self._pq.min()[1]

In [44]:
queue = PQFIFO() 
queue.enqueue("a")
queue.enqueue("b")
queue.enqueue("c")
queue.enqueue("d")

print(queue.top())
print(queue.dequeue())
print(queue.dequeue())

a
a
b


C-9.28 Professor Idle suggests the following solution to the previous problem.Whenever an item is inserted into the queue, it is assigned a key that is equal to the current size of the queue. Does such a strategy result in FIFO semantics? Prove that it is so or provide a counterexample.

no, think about the case when we do at least two deletions, the newely added element will have the same key as an exisiting element, which will cause a problem since we will not know where to put this one. 

C-9.29 Reimplement the SortedPriorityQueue using a Python list. Make sure to maintain remove min’s O(1) performance.

In [47]:
class SortedPQ(PriorityQueueBase): 
    def __init__(self):
        self._data = []

    def __len__(self):
        return len(self._data)

    def add(self, key, value):
        new = self._Item(key, value)
        self._data.append(new)
        idx = len(self._data)-1
        while idx >= 0 and self._data[idx]._key>self._data[idx-1]._key:
            self._data[idx], self._data[idx-1] = self._data[idx-1], self._data[idx]
            idx -= 1 
            
    def min(self):
        if self.is_empty():
            raise Empty("Empty Priority Queue")
        min_item = self._data[-1]
        return (min_item._key, min_item._value)

    def remove_min(self):
        if self.is_empty():
            raise Empty("Empty Priority Queue")
        min_item = self._data.pop(-1)
        return (min_item._key, min_item._value)


In [48]:
pq = SortedPQ()
pq.add(5, "a")
pq.add(2, "c")
pq.add(4, "d")
pq.add(1, "e")
print(pq)
print(pq.min())
print(pq.remove_min())
print(pq.min())
print(pq.remove_min())
print(pq.min())

(1, 'e')
(1, 'e')
(2, 'c')
(2, 'c')
(4, 'd')


* C-9.30 Give a nonrecursive implementation of the upheap method for the class
HeapPriorityQueue.
* C-9.31 Give a nonrecursive implementation of the downheap method for the class HeapPriorityQueue.

#### Full non-recursive heap data structure 
##### We'll implement a max-heap because we already have a min-heap 

In [49]:
class HeapPQ(PriorityQueueBase):
    def __init__(self, contents = ()):
        self._data = [self._Item(k, v) for k, v in contents]
            
    def __len__(self):
        return len(self._data) 
    
    def _heapify(self):
        leafs_start = self._parent(len(self._data)-1)
        for i in range(leafs_start, -1, -1):
            self._downheap(i) 
    
    def add(self, key, value):
        self._data.append(self_Item(key, value))
        self._upheap(len(self._data)-1) 
    
    def min(self):
        if len(self._data) == 0:
            raise Empty('Prioirty Queue is Empty!')
        mini = self._data[0]
        return (mini._key, mini._value)
    
    def remove_min(self):
        if len(self._data) == 0:
            raise Empty('Prioiry Queue is Empty!')
        self._swap(0, len(self._data)-1)
        mini = self._data.pop()
        self._downheap(0)
        return (mini._key, mini._value) 
    
    def _parent(self, j):
        return (j-1)//2 
    
    def _left(self, j):
        return 2*j+1 
    
    def _right(self, j):
        return 2*j+2 
    
    def _has_left(self, j):
        return self._left(j)<len(self._data) 
    
    def _has_right(self, j):
        return self._right(j)<len(self._data) 
    
    def _swap(self, i, j):
        self._data[i], self._data[j] = self._data[j], self._data[i] 
    
    def _upheap(self, j):
        while j>0 and self._data[self._parent(j)]._key < self._data[j]._key:
            self._swap(j, self._parent(j))
            j = self._parent(j) 
    
    def _downheap(self, j):
        while j<len(self._data) and self._has_left(j):
            larger = self._left(j)
            if self._has_right(j): 
                right = self._right(j)
                if self._data[right]._key>self._data[larger]._key:
                    larger = right
            if self._data[j] < self._data[larger]: 
                self._swap(j, larger)
            j = larger 
    
    def __repr__(self): 
        return ''.join("("+str(item._key)+","+str(item._value)+")" for item in self._data) 

In [50]:
pq = HeapPQ([(1, "a"), (2, "c"), (7,"b"), (4,"e"), (9, 'f'), (8, 'q')])
#pq._upheap(1)
print(pq)
#pq._downheap(0)
print(pq)
pq._heapify() 
print(pq)

(1,a)(2,c)(7,b)(4,e)(9,f)(8,q)
(1,a)(2,c)(7,b)(4,e)(9,f)(8,q)
(9,f)(4,e)(8,q)(1,a)(2,c)(7,b)


In [51]:
#(9,f)(4,e)(8,q)(1,a)(2,c)(7,b)

C-9.32 Assume that we are using a linked representation of a complete binary tree T , and an extra reference to the last node of that tree. Show how to update the reference to the last node after operations add or remove min in O(logn) time, where n is the current number of nodes of T . Be sure and handle all possible cases, as illustrated in Figure 9.12.

- check if the node is on the right to its parent. if so return the left of the parent. 
- Else: traverse up to the root node, this node has only two cases: 
    - either it is the left most node with a other nodes in the same level to the right to it, and we can tell that if we traverse up to the root and found ourselves in the right subtree of the root. 
    - Otherwiese, (meaning that we are on the left subtree of the root) we can get the leftmost node on the upper level, by going down left from the root. 

C-9.33 When using a linked-tree representation for a heap, an alternative method for finding the last node during an insertion in a heap T is to store, in the last node and each leaf node of T , a reference to the leaf node immediately to its right (wrapping to the first node in the next lower level for the rightmost leaf node). Show how to maintain such references in O(1) time per operation of the priority queue ADT assuming that T is implemented with a linked structure.

- I have a slightly smarter way of doing that, instead of strong a reference, we can store a binary number based on the location of the item in the binary tree. and we can easily use this number to tell if the node has siblings, or if it's the only leaf in a last level of a binary tree. and we can also use it to locate the next node to remove in a complete bianry tree. 


C-9.34 We can represent a path from the root to a given node of a binary tree by means of a binary string, where 0 means “go to the left child” and 1 means “go to the right child.” For example, the path from the root to the node storing (8,W ) in the heap of Figure 9.12a is represented by “101.”Design an O(logn)-time algorithm for finding the last node of a complete binary tree with n nodes, based on the above representation. Show how this algorithm can be used in the implementation of a complete binary tree by means of a linked structure that does not keep a reference to the last node

* the path to a node is represented by a binary string. To go to the first left most node after removing the current leftmost is to look at the representation of the current node, we go to the node with representation = representation of current node -1. if current node has all zeros representation, we do a two complement and take out the extra digit (or simply go to the node with all 1s but string representation that's 1 bit shorter than the representation for the current node). 

C-9.35 Given a heap T and a key k, give an algorithm to compute all the entries in T having a key less than or equal to k. For example, given the heap of Figure 9.12a and query k = 7, the algorithm should report the entries with keys 2, 4, 5, 6, and 7 (but not necessarily in this order). Your algorithm should run in time proportional to the number of entries returned, and should not modify the heap

- traverse the heap from the top, check the left and the right of the item, if the item you find is less than or equal to the key, return that number, and only check it's children if the key value of that item is less than the key. Hence you are guaranteed you are not going to go very deep, this is approxiamtely O(K). 

C-9.38 Suppose two binary trees, T1 and T2, hold entries satisfying the heap-order property (but not necessarily the complete binary tree property). Describe a method for combining T1 and T2 into a binary tree T , whose nodes hold the union of the entries in T1 and T2 and also satisfy the heap-order property. Your algorithm should run in time O(h1 + h2) where h1 and h2 are the respective heights of T1 and T2.

- if T1, and T2 are normal heaps, this can be done in linear time O(n1+n2), we put all the elements in one array and heapify. But to do that in log time this requires the two heaps to be a binomial heaps which are not covered in this chapter (advanced data strucutres). 

C-9.39 Implement a heappushpop method for the HeapPriorityQueue class, with semantics akin to that described for the heapq module in Section 9.3.7

In [52]:
class pushpopHeapPriorityQueue(HeapPriorityQueue): 
    def heappushpop(self, key, value):
        e = self._Item(key, value)
        mini_key, mini_value = self.min()
        if e._key < mini_key:
            return (e._key, e._value) 
        else:
            mini = self._data[0]
            self._data[0] = e
            self._downheap(0)
            return (mini._key, mini._value) 

       

In [53]:
pq = pushpopHeapPriorityQueue()
pq.add(5, "a")
pq.add(2, "c")
pq.add(4, "d")
pq.add(1, "e")
print(pq)
print(pq.heappushpop(7, 'zz'))
print(pq)

(1,e)(2,c)(4,d)(5,a)
(1, 'e')
(2,c)(5,a)(4,d)(7,zz)


C-9.40 Implement a heapreplace method for the HeapPriorityQueue class, with semantics akin to that described for the heapq module in Section 9.3.7.

In [54]:
class heapreplaceHeapPriorityQueue(HeapPriorityQueue):
    def heapreplace(self, key, value):
        mini = self.min()
        self._data[0] = self._Item(key, value)
        self._downheap(0)
        return mini

In [56]:
pq = heapreplaceHeapPriorityQueue()
pq.add(5, "a")
pq.add(2, "c")
pq.add(4, "d")
pq.add(1, "e")
print(pq)
print(pq.heapreplace(0, 'zz'))
print(pq)

(1,e)(2,c)(4,d)(5,a)
(1, 'e')
(0,zz)(2,c)(4,d)(5,a)


C-9.41 Tamarindo Airlines wants to give a first-class upgrade coupon to their top logn frequent flyers, based on the number of miles accumulated, where n is the total number of the airlines’ frequent flyers. The algorithm they currently use, which runs in O(nlog n) time, sorts the flyers by the number of miles flown and then scans the sorted list to pick the top logn flyers. Describe an algorithm that identifies the top logn flyers in O(n) time

- put all customers in an array and do heapify. O(n)
- take first log(n) items = O(log(n) * log(n)) = O(log(2n)) 
- total will be O(n + log(2n)) => O(n) 

C-9.42 Explain how the k largest elements from an unordered collection of size n can be found in time O(n + k logn) using a maximum-oriented heap.

- Heapify first O(n)
- take first k elements in O(klog(n)),
- total O(n + klog(N))

C-9.43 Explain how the k largest elements from an unordered collection of size n can be found in time O(nlogk) using O(k) auxiliary space.

- allocate a space with capacity k. 
- needing k-largest element -> use min heap. 
- pass through the n elements: 
    - if element smaller than top of heap, go to next. 
    - if element larger than top of heap, push top of heap, insert element and downheap
- resulting array has the top k elements 
    O(nlog(k)) + extra space O(k) 
    
"no restriction on the order of the elements" 

C-9.44 Given a class, PriorityQueue, that implements the minimum-oriented priority queue ADT, provide an implementation of a MaxPriorityQueue class that adapts to provide a maximum-oriented abstraction with methods add, max, and remove max. Your implementation should not make any assumption about the internal workings of the original PriorityQueue class, nor the type of keys that might be used.

In [65]:
class maxHeapPriorityQueue(HeapPriorityQueue):
    class _Item(HeapPriorityQueue._Item):
        def __lt__(self, other):
            return not super().__lt__(other)

    def max(self):
        return super().min() 
    
    def remove_max(self):
        return super().remove_min()
    
    def add(self, key, value):
        return super().add(key, value)

In [69]:
pq = maxHeapPriorityQueue()
pq.add(5, "a")
pq.add(2, "c")
pq.add(4, "d")
pq.add(1, "e")
print(pq)

print(pq.remove_max())
print(pq.remove_max())
print(pq.remove_max())
print(pq.remove_max())

(5,a)(2,c)(4,d)(1,e)
(5, 'a')
(4, 'd')
(2, 'c')
(1, 'e')


C-9.45 Write a key function for nonnegative integers that determines order based on the number of 1’s in each integer’s binary expansion.

In [70]:
def key_fun(num):
    return sum(int(char) for char in bin(num)[2:])

C-9.46 Give an alternative implementation of the pq_sort function, from Code Fragment 9.7, that accepts a key function as an optional parameter.

In [74]:
def edited_pq_sort(C):
    n = len(C)
    P = PriorityQueue()
    for j in range(n):
        element = C.delete(C.first())
        P.add(element, key_fun(element))
    for j in range(n):
        (k, v) = P.remove_min()
        C.add_last(v)

C-9.47 Describe an in-place version of the selection-sort algorithm for an array that uses only O(1) space for instance variables in addition to the array.

In [127]:
arr = [6,3,1,13,8,2,9,17]
loc = 0 
for i in range(len(arr)):
    minimum = arr[i]
    for j in range(i, len(arr)):
        if arr[j]<=minimum:
            minimum = arr[j]
            loc = j
    arr[i], arr[loc] = arr[loc], arr[i]
print(arr)

[1, 2, 3, 6, 8, 9, 13, 17]


C-9.48 Assuming the input to the sorting problem is given in an array A, describe how to implement the insertion-sort algorithm using only the array A and at most a constant number of additional variables.

In [128]:
arr = [6,3,1,13,8,2,9,17]
for i in range(1, len(arr)):
    j = i
    while(j>0):
        if arr[j]<arr[j-1]:
            arr[j], arr[j-1] = arr[j-1], arr[j]
        j-=1
print(arr)

[1, 2, 3, 6, 8, 9, 13, 17]


C-9.49 Give an alternate description of the in-place heap-sort algorithm using the standard minimum-oriented priority queue (instead of a maximumoriented one)

- you can do the same operation if you add_first your elements in stead of add_last, check the code fragment 9.7 in the book. 